In [146]:
import numpy as np
from icecream import ic

In [147]:
problem = np.load('problem_0.npz')
x = problem['x']
y = problem['y']
x.shape

(2, 1000)

In [ ]:
from gxgp import Node

input = x
labels = y

#Single input and output
print(input.shape, labels.shape)
#Print a column of input
print("INPUT")
print(input[:,0])
print("LABEL")
print(labels[0])
#Functional set
from operations.operations_dict import basic_function_set, complex_function_set

# Terminal set
terminal_list = ['x' + str(i) for i in range(input.shape[0])]

print(terminal_list)

(2, 1000) (1000,)
INPUT
[ 2.37857096 -0.74681473]
LABEL
2.242710018984468
['x0', 'x1']


Tree Generation  

In [149]:
import random


def generate_random_tree(max_depth, basic_function_set, complex_function_set, pc,terminal_set):
    if max_depth == 0 or (random.random() < 0.3):  # Terminale con probabilità
        terminal = random.choice(terminal_set)
        return Node(terminal)
    else:
        if random.random() < pc:
            func = random.choice(list(complex_function_set.keys()))
            num_children = complex_function_set[func].__code__.co_argcount  # Numero di argomenti della funzione
            children = [generate_random_tree(max_depth - 1, basic_function_set, complex_function_set, pc,terminal_set)
                        for _ in range(num_children)]
            return Node(complex_function_set[func], children,name=func)
        else:
            func = random.choice(list(basic_function_set.keys()))
            num_children = basic_function_set[func].__code__.co_argcount  # Numero di argomenti della funzione
            children = [generate_random_tree(max_depth - 1, basic_function_set, complex_function_set, pc,terminal_set)
                        for _ in range(num_children)]
            return Node(basic_function_set[func], children,name=func)
        


Generation

In [ ]:
def initialized_tree(max_depth, basic_function_set, complex_function_set, complex_func_probability,terminal_set):
    return generate_random_tree(max_depth, basic_function_set, complex_function_set, pc,terminal_set)

initialized = generate_random_tree(6, basic_function_set,complex_function_set, 0.05, terminal_list)


C:\Users\Fabio\AppData\Local\Temp\ipykernel_16888\110250009.py:2: UserWarning: Drawing not available (Key backend: 'module://matplotlib_inline.backend_inline' is not a valid value for backend; supported values are ['gtk3agg', 'gtk3cairo', 'gtk4agg', 'gtk4cairo', 'macosx', 'nbagg', 'notebook', 'qtagg', 'qtcairo', 'qt5agg', 'qt5cairo', 'tkagg', 'tkcairo', 'webagg', 'wx', 'wxagg', 'wxcairo', 'agg', 'cairo', 'pdf', 'pgf', 'ps', 'svg', 'template'])
  initialized.draw()


Create input formatted

In [151]:
vars = {'x'+str(i): input[:,0][i] for i in range(input.shape[0])}
initialized(**vars)

np.float64(2.9363031974471028)

In [ ]:
# Fitness
def fitness(mytree, vars, labels):
    return 100*np.square(labels - mytree(**vars)).sum()/len(labels)

MSE (train): 1197.06


In [ ]:
from gxgp.gp_common import xover_swap_subtree
# Xover
def xover(tree1, tree2):
    return xover_swap_subtree(tree1, tree2)

## Training